In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
)
from theme import apply_theme
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [ ]:
WAVE_COLOR = [
    '#8B98A4', # '#BA4338', # early
    '#1E65A4' # late
]
AXIS_SHOW = alt.Axis(grid=True, labels=True, ticks=True, domain=True, tickMinStep=1)
AXIS_HIDE = alt.Axis(grid=True, labels=False, ticks=False, domain=True)

In [ ]:
df = pd.read_csv(join("..", "data", "1.1.resurgence", "demographics", "demographic_stats.csv"))

# List of values to get a better sense on the data
print(
    # Singapore data shouldn't be included here
    # demo_df.country.unique().tolist(),
    df.group.unique().tolist(),
    # demo_df.country_wave.unique().tolist()
) 

df

In [ ]:
ageGroups = ['0-25', '26-49', '50-69', '70-79', '80+']
sexGroups = ['Female', 'Male']
raceGroups = ['White', 'Black']

#### Rename columns and values so that it looks more organized

In [ ]:
cdf = df.copy()
cdf = cdf.rename(columns={
    'p.all': 'p_all', 
    'n.all': 'n_all'
})
cdf.group = df.group.apply(
    lambda x: {
        '00to25': '0-25',
        '26to49': '26-49',
        '50to69': '50-69',
        '70to79': '70-79',
        '80plus': '80+',
        'female': 'Female',
        'male': 'Male',
        'white': 'White',
        'black': 'Black',
        'other': 'Other'
    }[x]
)
cdf.wave = df.wave.apply(
    lambda x: {
        'early': 'Early',
        'late': 'Late'
    }[x]
)
cdf

#### Age

In [ ]:
agedf = cdf[cdf.group.isin(ageGroups)]
agedf

In [ ]:
"""
Age percentage bar chart
"""
agebar_base = alt.Chart(
    agedf
).encode(
    x=alt.X(
        'group:N', 
        title=None, 
        axis=AXIS_HIDE
    ),
    y=alt.Y('p_all:Q', axis=alt.Axis(format='.0%'), title="Percentage of Patients"),
    color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave")
).properties(
#     title={
#         "text": 'Age Groups by Wave',
#         "dx": 0,
#         "subtitle": get_visualization_subtitle(data_release='2021-01-25', num_sites='?'), 
#         "subtitleColor": "gray",
#     },
    width=280, height=200
)

agebar_early = agebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Early']}
).mark_bar(
    xOffset=-10,
    size=20, stroke='black'
)
agebar_late = agebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Late']}
).mark_bar(
    xOffset=10,
    size=20, stroke='black'
)

agebar = alt.layer(agebar_early, agebar_late)

"""
Age N size bar chart
"""
nagebar_base = alt.Chart(
    agedf
).encode(
    x=alt.X(
        'group:N', 
        title='Age', 
        axis=AXIS_SHOW
    ),
    y=alt.Y('n_all:Q', title="# of Patients"),
    color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave")
).properties(
    width=280, height=70
)

nagebar_early = nagebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Early']}
).mark_bar(
    xOffset=-10,
    size=20, stroke='black'
)
nagebar_late = nagebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Late']}
).mark_bar(
    xOffset=10,
    size=20, stroke='black'
)
nagebar = alt.layer(nagebar_early, nagebar_late)

# agebar = alt.vconcat(agebar, nagebar)

# agebar = apply_theme(
#     agebar,
#     legend_orient='top-left'
# )

# rule = agebar_base.mark_rule(color='grey').encode(
#     x='group:N',
#     size=alt.value(10)
# )

(agebar)

#### Sex

In [ ]:
sexdf = cdf[cdf.group.isin(sexGroups)]
sexdf

In [ ]:
"""
Visual parameters
"""
width = 140

"""
Sex percentage bar chart
"""
sexbar_base = alt.Chart(
    sexdf
).encode(
    x=alt.X(
        'group:N', 
        title=None, 
        axis=AXIS_HIDE
    ),
    y=alt.Y('p_all:Q', axis=alt.Axis(format='.0%', grid=True, labels=False, ticks=False, domain=True), title=None),
    color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave", legend=None)
).properties(
#     title={
#         "text": 'Sex Group by Wave',
#         "dx": 0,
#         "subtitle": get_visualization_subtitle(data_release='2021-01-25', num_sites='?'), 
#         "subtitleColor": "gray",
#     },
    width=width, height=200
)

sexbar_early = sexbar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Early']}
).mark_bar(
    xOffset=-10,
    size=20, stroke='black'
)
sexbar_late = sexbar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Late']}
).mark_bar(
    xOffset=10,
    size=20, stroke='black'
)

sexbar = alt.layer(sexbar_early, sexbar_late)

"""
Sex N size bar chart
"""
nsexbar_base = alt.Chart(
    sexdf
).encode(
    x=alt.X(
        'group:N', 
        title='Sex', 
        axis=AXIS_SHOW
    ),
    y=alt.Y('n_all:Q', title=None, axis=AXIS_HIDE),
    color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave")
).properties(
    width=width, height=70
)

nsexbar_early = nsexbar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Early']}
).mark_bar(
    xOffset=-10,
    size=20, stroke='black'
)
nsexbar_late = nsexbar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Late']}
).mark_bar(
    xOffset=10,
    size=20, stroke='black'
)
nsexbar = alt.layer(nsexbar_early, nsexbar_late)

# sexbar = alt.vconcat(sexbar, nsexbar)

# sexbar = apply_theme(
#     sexbar,
#     legend_orient='top-left'
# )

sexbar

#### Race

In [ ]:
racedf = cdf[cdf.group.isin(raceGroups)]
racedf

In [ ]:
"""
Visual parameters
"""
width = 140

"""
Race percentage bar chart
"""
racebar_base = alt.Chart(
    racedf
).encode(
    x=alt.X(
        'group:N', 
        title=None, 
        axis=AXIS_HIDE
    ),
    y=alt.Y('p_all:Q', axis=alt.Axis(format='.0%', grid=True, labels=False, ticks=False, domain=False), title=None),
    color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave", legend=None)
).properties(
#     title={
#         "text": 'Sex Group by Wave',
#         "dx": 0,
#         "subtitle": get_visualization_subtitle(data_release='2021-01-25', num_sites='?'), 
#         "subtitleColor": "gray",
#     },
    width=width, height=200
)

racebar_early = racebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Early']}
).mark_bar(
    xOffset=-10,
    size=20, stroke='black'
)
racebar_late = racebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Late']}
).mark_bar(
    xOffset=10,
    size=20, stroke='black'
)

racebar = alt.layer(racebar_early, racebar_late)

"""
Race N size bar chart
"""
nracebar_base = alt.Chart(
    racedf
).encode(
    x=alt.X(
        'group:N', 
        title='Race', 
        axis=AXIS_SHOW
    ),
    y=alt.Y('n_all:Q', title=None, axis=AXIS_HIDE),
    color=alt.Color("wave:N", scale=alt.Scale(domain=['Early', 'Late'], range=WAVE_COLOR), title="Wave")
).properties(
    width=width, height=70
)

nracebar_early = nracebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Early']}
).mark_bar(
    xOffset=-10,
    size=20, stroke='black'
)
nracebar_late = nracebar_base.transform_filter(
    {'field': 'wave', 'oneOf': ['Late']}
).mark_bar(
    xOffset=10,
    size=20, stroke='black'
)
nracebar = alt.layer(nracebar_early, nracebar_late)

# sexbar = alt.vconcat(sexbar, nsexbar)

# sexbar = apply_theme(
#     sexbar,
#     legend_orient='top-left'
# )

racebar

In [ ]:
final_chart = alt.hconcat(agebar, sexbar, racebar).resolve_scale(y='shared')

nfinal_chart = alt.hconcat(nagebar, nsexbar, nracebar).resolve_scale(y='shared')

final_chart = (
    (final_chart & nfinal_chart).properties(
        title={
            "text": 'Demographics by Wave',
            "dx": 80,
            "subtitle": get_visualization_subtitle(data_release='2021-01-25', num_sites='?'), 
            "subtitleColor": "gray",
        }
    )
)

final_chart = apply_theme(
    final_chart,
    title_anchor='start',
    legend_orient='top-left'
)
final_chart